# Mathematik für Biologiestudierende

Wintersemester 2025/26

27.01.2026

&copy; 2026 Prof. Dr. Rüdiger W. Braun 

# Wiederholung (interaktiv)

Gehen Sie auf die Website

* https://pingo.coactum.de

und geben Sie folgende Zugangsnummer ein

* **670719**

oder scannen Sie den QR-Code

![QR-Code](bilder/qr02.png)

# Themen

* Korrelation
* empirischer Korrelationskoeffizient
* Lineare Modelle
* Interpretation der Ausgabe
* Lineare Modelle mit mehreren erklärenden Variablen

In [ ]:
import numpy as np
np.set_printoptions(legacy='1.21')
import seaborn as sns
sns.set_theme()
sns.set_context('talk')
import pandas as pd
from scipy import stats

# Korrelation

* Eine Korrelation zwischen zwei Datensätzen ist eine gemeinsame oder gegenläufige Tendenz.
* Beispielsweise steigt der Blutdruck tendenziell mit dem Alter.
* Gemessen wird die Korrelation durch den empirischen Korrelationskoeffizienten.
* Der empirischen Korrelationskoeffizient beantwortet die Frage
 
>      Gibt es eine Korrelation?

### Beispiel für zwei unkorrelierte Größen

* formal ist es auch möglich, Regressionsplot für zwei unkorrelierte Größen auszurechnen

In [ ]:
zufall = pd.DataFrame()
zufall['Zufall1'] = stats.norm.rvs(size=30)
zufall['Zufall2'] = stats.norm.rvs(size=30)
zufall.head()

In [ ]:
sns.regplot(zufall, x='Zufall1', y='Zufall2');

# Empirischer Korrelationskoeffizient

* Kennzahl zur Überprüfung gemeinsamer Tendenz
* $s_x$ sei die Stichprobenstreuung der $x_j$ und $s_y$ die Stichprobenstreuung der $y_j$ 
* dann ist der *empirische Korrelationskoeffizient* gleich
$$      r = \frac{\text{covar}_{\text{emp}}(x,y)}{s_x \cdot s_y} $$

* Der Korrelationskoeffizient ist dimensionslos

### Beispiel "Zufall"

In [ ]:
zufall.cov()

In [ ]:
sx = zufall.Zufall1.std()
sx

In [ ]:
sy = zufall.Zufall2.std()
sy

In [ ]:
covar = zufall.Zufall1.cov(zufall.Zufall2)
covar

In [ ]:
r = covar / (sx*sy)
r

## Interpretation des empirischen Korrelationskoeffizienten

Der Korrelationskoeffizient zeigt an, ob zwei Datensätze eine gemeinsame Tendenz aufweisen

* wenn er nahe bei $1$ liegt, dann wachsen $x$ und $y$ gemeinsam (gemeinsame Tendenz)
* wenn er nahe bei $-1$ liegt, dann fällt $y$, wenn $x$ wächst (gegenläufige Tendenz)
* wenn er nahe bei $0$ liegt, dann gibt es kein gemeinsames Verhalten

* auch ein negativer Korrelationskoeffizient hat eine Bedeutung
* Beispiel:  Je weniger Pestizide ich im Garten ausbringe, desto mehr Bienen habe ich

## Berechnung mit `pandas`

In [ ]:
zufall.corr()

#### Beispiel: Blutdruckdaten

Wir hatten in der letzten Woche die Kovarianzen für die Blutdruckdaten bestimmt 

In [ ]:
blutdruck = pd.read_csv('blutdruckdaten.csv')
blutdruck.cov()

Jetzt bestimmen wir die Korrelationskoeffizienten

In [ ]:
blutdruck.corr()

* Alter und Blutdruck sind korreliert (aber nicht stark)
* die anderen Größen sind nicht korreliert

#### Beispielgraph mit sehr guter Korrelation

In [ ]:
df1 = pd.DataFrame()
P = stats.norm(0.005, 0.005)
stoerung = P.rvs(size=30)     # ganz kleine Störung
df1['Länge'] = np.arange(30)
df1['Breite'] = 5 - 0.002*df1.Länge + stoerung
df1.head()

In [ ]:
sns.regplot(df1, x='Länge', y='Breite');

In [ ]:
df1.corr()

# Regression zum Mittelwert

* Der Begriff *Regression* kommt von Francis Galton, einem Neffen von Charles Darwin
* Er hatte den auf der nächsten Folie gezeigten Datensatz analysiert
* Auf der $x$-Achse stehen die Größen der Väter in Zoll
* Auf der $y$-Achse stehen die Größen der Söhne in Zoll

In [ ]:
galton = pd.read_csv('galton.csv')
galton.head()

* Aufbereitung eines Datensatzes von Galton.  Die Aufbereitung stammt aus den Begleitdaten zum Buch "Linear Models with Python" von Faraway

In [ ]:
sns.regplot(galton, x='father', y='childHeight');

Die Steigung dieser Geraden ist positiv, aber deutlich kleiner als 1

In [ ]:
m = galton.father.cov(galton.childHeight) / galton.father.var()
m

## Regression zum Mittelwert:  Interpretation

* Die Söhne ungewöhnlich großer oder kleiner Väter sind im Mittel selbst zwar auch größer bzw. kleiner als der Mittelwert, aber diese Differenz ist kleiner als bei den Vätern
* Galton bezeichnet dies (ziemlich unfreundlich) als "Regression to mediocrity"
* Das gilt aber nur für die Individuen, nicht für die Population als Ganzes
* auch in der nächsten Generation gibt es wieder ungewöhnlich große Individuen, aber in anderen Familien

# Korrelation &ne; Kausalität

* Wenn der Korrelationskoeffizient von $x$ und $y$ nahe $0$ liegt, dann gibt es keinen kausalen Zusammenhang     zwischen ihnen (seltene nichtlineare Pänomene mal ausgenommen)
* Man kann aber im umgekehrten Fall von einem Korrelationskoeffizienten nahe bei $1$ nicht auf einen kausalen     Zusammenhang schließen

* Zum Beispiel nimmt seit Jahrzehnten in Deutschland sowohl die Zahl der Geburten als auch die Zahl der Störche ab
* Der kausale Zusammenhang ist aber umstritten

* Beispiel aus der Schlafforschung:  Mittagsschlafdauern über 90 Minuten sind ungesund 

<img src="https://imgs.xkcd.com/comics/correlation.png" alt="xkcd Cartoon 552" width="80%"  />

Quelle: http://xkcd.com/552

#### Beispiel: Bleibelastung im Gewebe von Ratten

* kontaminiertes Gelände: fange 10 Ratten
* unbelastetes Vergleichsgelände:  fange 10 Ratten
* für jede Ratte wird ihr Alter in Monaten und der Bleigehalt im Gewebe bestimmt

In [ ]:
ratten = pd.read_csv('ratten.csv')
ratten.head()

In [ ]:
kon = ratten[ratten.Gelände=='kontaminiert']
unk = ratten[ratten.Gelände=='unkontaminiert']

In [ ]:
kon.describe()

In [ ]:
unk.describe()

* Es gibt einen Unterschied in der Bleibelastung
* aber auch eine große Stichprobenstreuung

In [ ]:
stats.ttest_ind(unk.Belastung, kon.Belastung, alternative='less')

* Der Unterschied ist $\alpha = 0.05$ nicht signifikant.
* Es fällt aber auf, dass die Ratten von dem belasteten Gebiet im Mittel jünger als die anderen sind.
* Wir wollen das Alter herausrechnen

* Steigt die Bleibelastung mit dem Alter?

In [ ]:
kon.corr(numeric_only=True)   #  ohne die Option gibt es einen ValueEror

In [ ]:
unk.corr(numeric_only=True)

Wir zeigen beide Regressionen in einem Bild

In [ ]:
sns.lmplot(ratten, x='Alter', y='Belastung', hue='Gelände');

* die Gerade zu den Daten des kontaminierten Geländes liegt klar oberhalb der Geraden des unkontaminierten Geländes
* für 8 Monate alte Ratten sind dir Konfidenzintervalle disjunkt

* `sns.lmplot` vereint mehrere regplots, ähnlich wie das `sns.displot` tut
* es hat auch ähnliche Optionen

# Lineare Modelle

* eine lineare Funktion einer Variablen ist eine Funktion der Form
$$   y = m \cdot x + b
$$
* bei der linearen Regression besteht die Aufgabe darin, $m$ und $b$ zu bestimmen

Das Konzept des linearen Modells erweitert dieses Verfahren in doppelter Hinsicht

* Konfidenzintervalle für $m$ und $b$ werden bestimmt
* die Zielvariable $y$ kann von mehr als einer Größe abhängen

* Literatur: "Linear Models with Python" von Faraway
* Statsmodels: https://www.statsmodels.org/stable/user-guide.html

Wir beginnen mit den Blutdruckdaten

In [ ]:
sns.regplot(blutdruck, x='Alter', y='Blutdruck');

# Formulierung des Modells

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
formel = 'Blutdruck ~ Alter'

Das bedeutet:

* wir wollen den Blutdruck modellieren
* der Blutdruck ist die *abhängige Variable* (engl.  dependent)
* das Alter ist die *erklärende Variable*

In [ ]:
modell = smf.ols(formel, blutdruck)

* `ols`: ordinary least squares
* Lektion 25: die Regression ist "bestmöglich" in dem Sinn, dass
$$
    \sum_{j=1}^n (m \cdot x_j + b - y_j)^2 
$$
minimal wird
* daher der Name "Methode der kleinsten Quadrate"  (engl. "lest squares")

In [ ]:
res = modell.fit()

# Interpretation der Ausgabe

In [44]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Blutdruck   R-squared:                       0.701
Model:                            OLS   Adj. R-squared:                  0.690
Method:                 Least Squares   F-statistic:                     65.54
Date:                Tue, 27 Jan 2026   Prob (F-statistic):           8.17e-09
Time:                        07:56:37   Log-Likelihood:                -123.27
No. Observations:                  30   AIC:                             250.5
Df Residuals:                      28   BIC:                             253.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     80.3697      8.798      9.135      0.000      62.348      98.391
Alter          1.5081      0.186      8.096      0.000       1.127       1.890
==============================================================================
Omnibus:                        2.886   Durbin-Watson:                   2.401
Prob(Omnibus):                  0.236   Jarque-Bera (JB):                1.526
Skew:                           0.390   Prob(JB):                        0.466
Kurtosis:                       3.782   Cond. No.                         149.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* Ich werde die wichtigsten Daten aus dieser Ausgabe erkären

* in der ersten Zeile steht der Name der abhängigen Variablen

zum Vergleich:  wir hatten in Lektion 25 die lineare Regression zu Fuß gerechnet und für die Steigung den folgenden Wert erhalten:

In [ ]:
cov = blutdruck.Alter.cov(blutdruck.Blutdruck)
var_x = blutdruck.Alter.var()
m = cov / var_x
np.round(m, 4)

* Das ist genau die Zahl, die in der Spalte `coef` der Zeile `Alter` steht

* Der Wert für den Ordinatenabschnitt (engl: "intercept") war damals

In [ ]:
xq = blutdruck.Alter.mean()
yq = blutdruck.Blutdruck.mean()
b = yq - m*xq
np.round(b, 4)

* Das ist die Zahl, die in der Spalte `coef` und der Zeile `Intercept` steht

Wir schauen uns die Zeile `Alter` weiter an:

* Der Eintrag `P>|t|` bezeichnet den p-Wert für den zweiseitigen Test, dass `coef` ungleich 0 ist.
* In der Zeile "Alter" ist `coef` ist die Steigung der linearen Regression, also das $m$ 

* wenn die Nullhypothese $H_0=\{m=0\}$ nicht abgelehnt werden kann, dann bedeutet das, dass zum Signifikanzniveau $\alpha=0.05$ nicht nachgewiesen wurde, dass das Alter überhaupt einen Einfluss auf den Blutdruck hat

* Der Eintrag `t` ist der Wert der Teststatistik, aus dem der p-Wert bestimmt worden ist

# Konfidenzintervalle für die Koeffizienten

In [ ]:
res.summary()

* Die Einträge `[0.025` und `0.975]` geben die untere und die obere Vertrauensgrenze des Konfidenzintervalls zum Konfidenzniveau 0.95 für die Steigung an

* Variante:  99%-Konfidenzintervall
* Achtung:  Für Konfidenzintervall zum Konfidenzniveau $1-\alpha$ muss $\alpha$ eingegeben werden

In [ ]:
res.conf_int(alpha=.01)

* Der Wert für $m$ in der Formel für die lineare Regression liegt mit 99% Sicherheit zwischen 0.993 und 2.02

# Korrelationskoeffizienten

In Lektion 25 hatten wir den Korrelationskoeffizienten bestimmt

In [ ]:
r = 0.83705

In [ ]:
np.round(r**2, 3)

In [ ]:
res.summary()

* $r^2$ ist die Größe, die in `res.summary()` als `R-squared` auftaucht

# Mehrere erklärende Variablen

## Lineares Modell mit einer abhängigen und mehreren erklärenden Variablen


$$
   y = m_1 \cdot x_1 + m_2 \cdot x_2 + \dots + m_n \cdot x_n + b
$$

* $y$ ist die abhängige und die $x_i$ sind die erklärenden Variablen
* $b$ ist das `Intercept`
* $m_i$ sind die Steigungen

Beispiel:  Körpergröße der Söhne hängt von der Körpergröße von Vater und Mutter ab

In [ ]:
formel = 'childHeight ~ father + mother'

Diese Formel hat 3 Unbekannte:

* den Koeffizienten von `father`
* den Koeffizienten von `mother`
* den Ordinatenabschnitt

In [ ]:
modell = smf.ols(formel, galton)

In [ ]:
res = modell.fit()

In [ ]:
res.summary()

* Regressiongleichung:  childHeight = 0.4176 * father + 0.3288 * mother + 19.3128
* alle drei Koeffizienten haben statistisch signifikanten Einfluss

* Der Wert für $r^2$ beträgt 0.238

* Für das Modell, bei dem die Größe des Kindes nur über die Größe des Vaters modelliert wird, beträgt der Wert von $r^2$ nur 0.154
* In dem Modell, welches die Körpergrößen beider Eltern berücksichtigt, ist die Korrelation höher
* Das Modell erklärt die erklärende Variable besser

#### Beispiel: Hinzufügung einer Variablen ohne Einfluss

In [ ]:
galton['Kontonummer'] = stats.randint(1, 999999).rvs(size=481)

In [ ]:
formel = 'childHeight ~ father + mother + Kontonummer'
modell = smf.ols(formel, galton)
res = modell.fit()

In [ ]:
res.summary()

* Die Kontonummer hat keinen signifikanten Einfluss
* Das erkennt man auch daran, dass 0 im Konfidenzintervall liegt